<a href="https://colab.research.google.com/github/shardul28/political-stance-detection/blob/Mscproject/MODEL%20FML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!pip install mit-news-classify


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
import pandas as pd
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
import pickle
from mitnewsclassify import tfidf as mitx
from nltk.corpus import stopwords
from tqdm import tqdm
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict,Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
cd /content/drive/MyDrive/project_data

/content/drive/MyDrive/project_data


# MODEL


In [6]:
infile = open('/content/drive/MyDrive/project_data/clean_data.pkl','rb')
df = pickle.load(infile)

In [7]:
df

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,clean_content,text_lemmatized,clean_text_lemmatized,clean_content_with_Stopwords
0,"Army Officer, 4 Others Killed In Encounter In ...",https://www.ndtv.com/india-news/army-officer-4...,"Army Officer, 4 Personnel Killed In Encounter ...",an army officer and four other personnel were ...,3,9.0,63.0,ndtv,0,army officer personnel killed action encounter...,"[army, officer, personnel, killed, action, enc...","[army, officer, personnel, killed, action, enc...",an army officer and four other personnel were ...
1,"#AryanKhan Case: No Bail For Him Today, Next H...",https://ndtv.com/india-news/mumbai-drug-bust-c...,"Aryan Khan Case: No Bail For Him Today, Next H...",aryan khan did not get bail today for the thir...,3,5.0,44.0,ndtv,0,aryan khan bail time arrest week ago drugs cru...,"[aryan, khan, bail, time, arrest, week, ago, d...","[aryan, khan, bail, time, arrest, week, ago, d...",aryan khan did not get bail today for the thir...
2,RT @ndtvfeed: Ex-National Conference Leaders D...,https://www.ndtv.com/india-news/after-resignin...,"Ex-National Conference Leaders Devender Rana, ...",a day after resigning from the national confer...,1,2.0,0.0,ndtv,0,resigning national conference devender rana he...,"[resigning, national, conference, devender, ra...","[resigning, national, conference, devender, ra...",a day after resigning from the national confer...
3,"RT @ndtvfeed: US Has Lost AI Battle To China, ...",https://www.ndtv.com/world-news/us-has-lost-ar...,"US Has Lost AI Battle To China, Pentagon's Ex-...",china has won the artificial intelligence batt...,0,5.0,0.0,ndtv,0,china artificial intelligence battle united st...,"[china, artificial, intelligence, battle, unit...","[china, artificial, intelligence, battle, unit...",china has won the artificial intelligence batt...
4,RT @ndtvfeed: A Look At Inspiring Messages On ...,https://www.ndtv.com/india-news/on-internation...,International Day Of The Girl Child 2021: Lead...,various world leaders and un agencies posted w...,3,3.0,0.0,ndtv,0,various world leaders agencies posted wishes m...,"[various, world, leader, agency, posted, wish,...","[various, world, leader, agency, posted, wish,...",various world leaders and un agencies posted w...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2145,Kerala HC quashes state govt order of reducing...,https://www.republicworld.com/india-news/law-a...,Kerala HC Quashes State Govt Order Of Reducing...,the kerala high court quashed the state govern...,3,4.0,21.0,republictv,1,kerala high court quashed government order mon...,"[kerala, high, court, quashed, government, ord...","[kerala, high, court, quashed, government, ord...",the kerala high court quashed the state govern...
2146,Lakhimpur Kheri violence: Yogi Adityanath hold...,https://www.republicworld.com/india-news/gener...,Lakhimpur Kheri Violence: Yogi Adityanath Hold...,chief minister yogi adityanath on monday held ...,1,7.0,72.0,republictv,1,chief minister yogi adityanath monday held eme...,"[chief, minister, yogi, adityanath, monday, he...","[chief, minister, yogi, adityanath, monday, he...",chief minister yogi adityanath on monday held ...
2147,"IPL 2021: Know how KKR, PBKS, RR, &amp; MI can...",https://www.republicworld.com/sports-news/cric...,"IPL 2021: Know How KKR, PBKS, RR, & MI Can Qua...",the ongoing edition of the indian premier leag...,3,4.0,20.0,republictv,1,ongoing edition indian premier league currentl...,"[ongoing, edition, indian, premier, league, cu...","[ongoing, edition, indian, premier, league, cu...",the ongoing edition of the indian premier leag...
2148,Lakhimpur clash: Priyanka Gandhi protests deta...,https://www.republicworld.com/india-news/polit...,Lakhimpur Clash: Priyanka Gandhi Protests Deta...,"in a mark of silent protest, congress general ...",0,4.0,24.0,republictv,1,mark silent protest congress general secretary...,"[mark, silent, protest, congress, general, sec...","[mark, silent, pro

In [8]:
df['clean_content']=""
for i in tqdm(range(len(df['clean_text_lemmatized']))):
    x=df['clean_text_lemmatized'][i]
    review=" ".join(x)
    df['clean_content'][i] = review

  0%|          | 0/2150 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 2150/2150 [00:01<00:00, 1723.74it/s]


In [9]:
df

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,clean_content,text_lemmatized,clean_text_lemmatized,clean_content_with_Stopwords
0,"Army Officer, 4 Others Killed In Encounter In ...",https://www.ndtv.com/india-news/army-officer-4...,"Army Officer, 4 Personnel Killed In Encounter ...",an army officer and four other personnel were ...,3,9.0,63.0,ndtv,0,army officer personnel killed action encounter...,"[army, officer, personnel, killed, action, enc...","[army, officer, personnel, killed, action, enc...",an army officer and four other personnel were ...
1,"#AryanKhan Case: No Bail For Him Today, Next H...",https://ndtv.com/india-news/mumbai-drug-bust-c...,"Aryan Khan Case: No Bail For Him Today, Next H...",aryan khan did not get bail today for the thir...,3,5.0,44.0,ndtv,0,aryan khan bail time arrest week ago drug crui...,"[aryan, khan, bail, time, arrest, week, ago, d...","[aryan, khan, bail, time, arrest, week, ago, d...",aryan khan did not get bail today for the thir...
2,RT @ndtvfeed: Ex-National Conference Leaders D...,https://www.ndtv.com/india-news/after-resignin...,"Ex-National Conference Leaders Devender Rana, ...",a day after resigning from the national confer...,1,2.0,0.0,ndtv,0,resigning national conference devender rana he...,"[resigning, national, conference, devender, ra...","[resigning, national, conference, devender, ra...",a day after resigning from the national confer...
3,"RT @ndtvfeed: US Has Lost AI Battle To China, ...",https://www.ndtv.com/world-news/us-has-lost-ar...,"US Has Lost AI Battle To China, Pentagon's Ex-...",china has won the artificial intelligence batt...,0,5.0,0.0,ndtv,0,china artificial intelligence battle united he...,"[china, artificial, intelligence, battle, unit...","[china, artificial, intelligence, battle, unit...",china has won the artificial intelligence batt...
4,RT @ndtvfeed: A Look At Inspiring Messages On ...,https://www.ndtv.com/india-news/on-internation...,International Day Of The Girl Child 2021: Lead...,various world leaders and un agencies posted w...,3,3.0,0.0,ndtv,0,various world leader agency posted wish messag...,"[various, world, leader, agency, posted, wish,...","[various, world, leader, agency, posted, wish,...",various world leaders and un agencies posted w...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2145,Kerala HC quashes state govt order of reducing...,https://www.republicworld.com/india-news/law-a...,Kerala HC Quashes State Govt Order Of Reducing...,the kerala high court quashed the state govern...,3,4.0,21.0,republictv,1,kerala high court quashed government order mon...,"[kerala, high, court, quashed, government, ord...","[kerala, high, court, quashed, government, ord...",the kerala high court quashed the state govern...
2146,Lakhimpur Kheri violence: Yogi Adityanath hold...,https://www.republicworld.com/india-news/gener...,Lakhimpur Kheri Violence: Yogi Adityanath Hold...,chief minister yogi adityanath on monday held ...,1,7.0,72.0,republictv,1,chief minister yogi adityanath monday held eme...,"[chief, minister, yogi, adityanath, monday, he...","[chief, minister, yogi, adityanath, monday, he...",chief minister yogi adityanath on monday held ...
2147,"IPL 2021: Know how KKR, PBKS, RR, &amp; MI can...",https://www.republicworld.com/sports-news/cric...,"IPL 2021: Know How KKR, PBKS, RR, & MI Can Qua...",the ongoing edition of the indian premier leag...,3,4.0,20.0,republictv,1,ongoing edition indian premier league currentl...,"[ongoing, edition, indian, premier, league, cu...","[ongoing, edition, indian, premier, league, cu...",the ongoing edition of the indian premier leag...
2148,Lakhimpur clash: Priyanka Gandhi protests deta...,https://www.republicworld.com/india-news/polit...,Lakhimpur Clash: Priyanka Gandhi Protests Deta...,"in a mark of silent protest, congress general ...",0,4.0,24.0,republictv,1,mark silent protest congress general secretary...,"[mark, silent, protest, congress, general, sec...","[mark, silent, pro

In [10]:
# from keras.preprocessing.sequence import pad_sequences
# max_features = 30000
# maxlen = 25
# X = df['clean_text_lemmatized']
# X = pad_sequences(X, maxlen = maxlen)
# y=df['bias']

In [11]:
for x in df['clean_text_lemmatized'][2]:
   if(x=='day'):
      print(x)

In [12]:
for i in range(len(df)):
  for x in df['clean_text_lemmatized'][i]:
    if x=='wa':
      print(i)
      


In [13]:
df['content'][0]

"an army officer and four other personnel were killed in action during an encounter between security forces and terrorists in jammu and kashmir's poonch district on monday, officials said. an anti-insurgency operation was launched in a village close to dera ki gali in surankote in the early hours following intelligence inputs about the presence of heavily-armed terrorists there, they said. the hiding terrorists fired on the search parties resulting in fierce gunfight and critical injuries to a junior commissioned officer or jco and four other soldiers. they died from their injuries. the encounter with the terrorists was going on and further details are awaited, a defence spokesperson said. officials said there were reports about the presence of a group of heavily-armed terrorists in the chamrer forest, after they managed to sneak from across the line of control (loc). reinforcements have been rushed to the area to plug all escape routes of the terrorists, they said. (with inputs from p

In [14]:
#df['clean_text_lemmatized'][0]

In [15]:
//

SyntaxError: ignored

In [18]:
for i in range(len(df)):
  for x in df['clean_text_lemmatized'][i]:
    if str(x)=='said':
      print('here',i)


In [19]:
center_with_bias_words = df.loc[df['content'].str.contains('shiv sena|shivsena|aap|bjp|congress', case=False)]
center_without_bias_words = df.loc[~df['content'].str.contains('shiv sena|shivsena|aap|bjp|congress', case=False)]

center_without_bias_words = center_without_bias_words.loc[center_without_bias_words['bias']==3]

center_with_bias_words = center_with_bias_words.loc[center_with_bias_words['bias']==3]
right = df.loc[df['bias']==1]
left = df.loc[df['bias']==0]
#center = df.loc[df['bias']==3]

#df2=pd.concat([left,right,center_with_bias_words])


In [20]:
print('right',len(right))
print('left',len(left))
print('center_without_bias_words',len(center_without_bias_words))
print('center_with_bias_words',len(center_with_bias_words))



right 184
left 263
center_without_bias_words 1466
center_with_bias_words 237


In [21]:
left=left.head(190)

In [22]:
center_with_bias_words=center_with_bias_words.head(200)

In [23]:
center_without_bias_words=center_without_bias_words.head(800)


In [24]:
print('right',len(right))
print('left',len(left))
print('center_without_bias_words',len(center_without_bias_words))
print('center_with_bias_words',len(center_with_bias_words))



right 184
left 190
center_without_bias_words 800
center_with_bias_words 200


In [25]:
right['clean_content'][2]

'resigning national conference devender rana head jammu division party leader surjit singh slathia joined bjp monday mr rana mr slathia joined bjp headquarters new delhi presence union minister dharmendra pradhan hardeep singh puri jitendra singh mr rana mla younger brother union minister jitendra singh served political advisor jammu kashmir chief minister national conference nc leader omar abdullah bjp general secretary charge jammu kashmir tarun chugh jammu kashmir unit chief ravinder raina welcomed party mr rana advocating jammu declaration joint declaration political social business organisation primarily demanding restoration statehood jammu region jammu kashmir centre revoked erstwhile jammu kashmir special status bifurcated union territory jammu kashmir ladakh'

In [26]:
df2=pd.concat([left,right,center_without_bias_words])#,center_with_bias_words


In [27]:
#REINDEXING
indx=[]
for i in range(len(df2)):
  indx.append(i)
df2.index=indx

In [28]:
# ## Divide the dataset into Train and Test
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=2)

In [29]:
######NaiiveBayes without Feature Selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score,classification_report,accuracy_score,confusion_matrix
from sklearn.naive_bayes import GaussianNB
## TFidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [30]:
X = df2['clean_content']
y=df2['bias']


In [31]:
from nltk.tokenize import word_tokenize
all_words = []
for message in df2['clean_content']:
      words = word_tokenize(message)
      for w in words:
          all_words.append(w)
all_words = nltk.FreqDist(all_words)

In [32]:
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))

Number of words: 17452
Most common words: [('minister', 1340), ('government', 1240), ('people', 1185), ('farmer', 1108), ('police', 1015), ('mr', 924), ('court', 827), ('chief', 809), ('country', 794), ('party', 751), ('case', 741), ('congress', 714), ('modi', 708), ('leader', 692), ('bjp', 663)]


In [ ]:
///

In [80]:
## TFidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(max_features=35,ngram_range=(1,4))

In [81]:
X=tfidf_v.fit_transform(df2['clean_content']).toarray()


In [82]:
X

array([[0.        , 0.        , 0.53795109, ..., 0.27512482, 0.57218265,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.24035146],
       [0.25787541, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.50659823, 0.        , ..., 0.        , 0.        ,
        0.52668961]])

In [83]:
## Divide the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33 ,random_state=50)

In [84]:
count_df = pd.DataFrame(X_train, columns=tfidf_v.get_feature_names())
count_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,bjp,case,chief,chief minister,congress,country,court,covid,delhi,farmer,...,people,police,power,pradesh,singh,son,sunday,time,told,union
0,0.0,0.245496,0.000000,0.0,0.0,0.236553,0.000000,0.0,0.0,0.000000,...,0.189366,0.504235,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0
1,0.0,0.000000,0.291925,0.0,0.0,0.000000,0.747274,0.0,0.0,0.000000,...,0.235740,0.000000,0.0,0.000000,0.0,0.353494,0.000000,0.0,0.310501,0.0
2,0.0,0.000000,0.000000,0.0,0.0,0.312822,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.380414,0.0,0.000000,0.640098,0.0,0.000000,0.0
3,0.0,0.000000,0.455895,0.0,0.0,0.459888,0.000000,0.0,0.0,0.594029,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0
4,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.921777,0.0,0.0,0.000000,...,0.387720,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0


In [85]:
y_train.value_counts()

3    530
0    129
1    127
Name: bias, dtype: int64

In [86]:
y_test.value_counts()


3    270
0     61
1     57
Name: bias, dtype: int64

In [87]:
//

SyntaxError: ignored

In [89]:
y_test.value_counts()

3    270
0     61
1     57
Name: bias, dtype: int64

In [88]:
from sklearn import model_selection, naive_bayes, svm
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train,y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(X_test)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, y_test)*100)
print(classification_report(y_test,predictions_NB))

Naive Bayes Accuracy Score ->  82.9896907216495
              precision    recall  f1-score   support

           0       0.59      0.70      0.64        61
           1       0.83      0.35      0.49        57
           3       0.89      0.96      0.92       270

    accuracy                           0.83       388
   macro avg       0.77      0.67      0.69       388
weighted avg       0.83      0.83      0.82       388



In [90]:
rf_classifier=RandomForestClassifier( max_features='auto', min_samples_leaf=4, min_samples_split=5, n_estimators=1000).fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)
y.value_counts()
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))


[[ 41  11   9]
 [ 16  25  16]
 [  6   1 263]]
0.8479381443298969
              precision    recall  f1-score   support

           0       0.65      0.67      0.66        61
           1       0.68      0.44      0.53        57
           3       0.91      0.97      0.94       270

    accuracy                           0.85       388
   macro avg       0.75      0.69      0.71       388
weighted avg       0.84      0.85      0.84       388



In [91]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier=RandomForestClassifier(n_estimators=300).fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)
y.value_counts()
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))


[[ 42  14   5]
 [ 14  31  12]
 [  4   6 260]]
0.8582474226804123
              precision    recall  f1-score   support

           0       0.70      0.69      0.69        61
           1       0.61      0.54      0.57        57
           3       0.94      0.96      0.95       270

    accuracy                           0.86       388
   macro avg       0.75      0.73      0.74       388
weighted avg       0.85      0.86      0.86       388



# RF  Manual Hyperparameter Tuning

In [ ]:
### Manual Hyperparameter Tuning
# {'criterion': 'entropy',
#  'max_depth': 100,
#  'max_features': 'sqrt',
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 600}
model=RandomForestClassifier(n_estimators=900,criterion='entropy',
                             max_features='sqrt',min_samples_leaf=1,min_samples_split=2,random_state=100).fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

In [ ]:
//

#RF WITH RANDOMSEARCH CV

RF WITH RANDOMSEARCH CV

In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(50, 500,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

In [ ]:
rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=10,cv=3,verbose=2,
                               random_state=50,n_jobs=-1)
### fit the randomized model
rf_randomcv.fit(X_train,y_train)

In [ ]:
rf_randomcv.best_params_

In [ ]:
best_random_grid=rf_randomcv.best_estimator_
from sklearn.metrics import accuracy_score
y_pred=best_random_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: \n {}".format(classification_report(y_test,y_pred)))

#RF WITH GridSearchCV


RF WITH GridSearchCV


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [rf_randomcv.best_params_['criterion']],
    'max_depth': [rf_randomcv.best_params_['max_depth']],
    'max_features': [rf_randomcv.best_params_['max_features']],
    'min_samples_leaf': [rf_randomcv.best_params_['min_samples_leaf'], 
                         rf_randomcv.best_params_['min_samples_leaf']+2, 
                         rf_randomcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rf_randomcv.best_params_['min_samples_split'] - 2,
                          rf_randomcv.best_params_['min_samples_split'] - 1,
                          rf_randomcv.best_params_['min_samples_split'], 
                          rf_randomcv.best_params_['min_samples_split'] +1,
                          rf_randomcv.best_params_['min_samples_split'] + 2],
    'n_estimators': [rf_randomcv.best_params_['n_estimators'] - 200, rf_randomcv.best_params_['n_estimators'] - 100, 
                     rf_randomcv.best_params_['n_estimators'], 
                     rf_randomcv.best_params_['n_estimators'] + 100, rf_randomcv.best_params_['n_estimators'] + 200]
}

print(param_grid)
#### Fit the grid_search to the data
rf=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)


In [ ]:
//

# RF with RandomizedSearchCV

In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(param)

# RF WITH TPOT OPTIMIZATION

In [ ]:
pip install tpot


In [ ]:
from tpot import TPOTClassifier

tpot_classifier = TPOTClassifier(generations= 3, population_size= 30, offspring_size= 10,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier':param}, 
                                 cv = 4, scoring = ['accuracy','recall','precision'])
tpot_classifier.fit(X_train,y_train)

In [ ]:
### Manual Hyperparameter Tuning
# {'criterion': 'entropy',
#  'max_depth': 100,
#  'max_features': 'sqrt',
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 600}
#model=RandomForestClassifier( criterion='gini', max_depth=340, max_features='auto', min_samples_leaf=2, min_samples_split=5, n_estimators=2000).fit(X_train,y_train)
model=RandomForestClassifier(criterion='entropy', max_depth=780, max_features='auto', min_samples_leaf=4, min_samples_split=5, n_estimators=1000).fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

In [ ]:
pip install optuna


In [ ]:
import optuna
import sklearn.svm
def objective(trial):

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 200, 2000,10)
        max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf,X_train,y_train, n_jobs=-1, cv=3).mean()

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
rf=RandomForestClassifier(n_estimators=330,max_depth=30)
rf.fit(X_train,y_train)

In [ ]:
y_pred=rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
text_clf = text_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred=text_clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: \n {}".format(classification_report(y_test,y_pred)))

# New Section